In [1]:
import os
import gc
import cv2
import copy
import time
import random
import string
import joblib
import tifffile
import numpy as np 
import pandas as pd 
import torch
from torch import nn
import seaborn as sns
from torchvision import models
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from torch.optim import lr_scheduler
import warnings
warnings.filterwarnings("ignore")
gc.enable()

In [2]:
debug = False
generate_new = True
test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
dirs = ["../input/mayo-clinic-strip-ai/train/", "../input/mayo-clinic-strip-ai/test/"]
test_df

,image_id,center_id,patient_id,image_num
0,006388_0,11,006388,0
1,008e5c_0,11,008e5c,0
2,00c058_0,11,00c058,0
3,01adc5_0,11,01adc5,0


In [3]:
try:
    os.mkdir("../test/")
except:
    pass
for i in tqdm(range(test_df.shape[0])):
    img_id = test_df.iloc[i].image_id
    try:
        sz = os.path.getsize(dirs[1] + img_id + ".tif")
    except:
        sz = 1000000000
    if(sz > 8e8):
        img = np.zeros((512,512,3), np.uint8)
    else:
        try:
            img = cv2.resize(tifffile.imread(dirs[1] + img_id + ".tif"), (512, 512))
        except:
            img = np.zeros((512,512,3), np.uint8)
    cv2.imwrite(f"../test/{img_id}.jpg", img)
    del img
    gc.collect()

  0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
class ImgDataset(Dataset):
    def __init__(self, df):
        self.df = df 
        self.train = 'label' in df.columns
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        if(generate_new):
            paths = ["../test/", "../train/"]
        else:
            paths = ["../input/jpg-images-strip-ai/test/", "../input/jpg-images-strip-ai/train/"]
        try:
            image = cv2.imread(paths[self.train] + self.df.iloc[index].image_id + ".jpg")
        except:
            image = np.zeros((512,512,3), np.uint8)
        label = 0
        try:
            if len(image.shape) == 5:
                image = image.squeeze().transpose(1, 2, 0)
            image = cv2.resize(image, (512, 512)).transpose(2, 0, 1)
        except:
            image = np.zeros((3, 512, 512))
        if(self.train):
            label = {"CE" : 0, "LAA": 1}[self.df.iloc[index].label]
        patient_id = self.df.iloc[index].patient_id
        return image, label, patient_id

In [5]:
def predict(model, dataloader):
    model.cuda()
    model.eval()
    dataloader = dataloader
    outputs = []
    s = nn.Softmax(dim=1)
    ids = []
    for item in tqdm(dataloader, leave=False):
        patient_id = item[2][0]
        try:
            images = item[0].cuda().float()
            ids.append(patient_id)
            output = model(images)
            outputs.append(s(output.cpu()[:,:2])[0].detach().numpy())
        except:
            ids.append(patient_id)
            outputs.append(s(torch.tensor([[1, 1]]).float())[0].detach().numpy())
    return np.array(outputs), ids

In [6]:
#model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b4', pretrained=True)
model = torch.jit.load('../input/cnnstripai/model.pth')
batch_size = 1
test_loader = DataLoader(
    ImgDataset(test_df), 
    batch_size=batch_size, 
    shuffle=False, 
    num_workers=1
)

In [7]:
anss, ids = predict(model, test_loader)

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
prob = pd.DataFrame({"CE" : anss[:,0], "LAA" : anss[:,1], "id" : ids}).groupby("id").mean()

In [9]:
submission = pd.read_csv("../input/mayo-clinic-strip-ai/sample_submission.csv")

In [10]:
submission.CE = prob.CE.to_list()
submission.LAA = prob.LAA.to_list()

In [11]:
submission

,patient_id,CE,LAA
0,006388,0.620316,0.379684
1,008e5c,0.524719,0.475281
2,00c058,0.282198,0.717802
3,01adc5,0.414263,0.585737


In [12]:
submission.to_csv("submission.csv", index = False)